In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 8.1MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [43.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu bioni

In [2]:
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium import common
from tqdm import tqdm
import urllib3
from parsing import get_html

In [3]:
urllib3.disable_warnings()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [4]:
base_url = 'https://market.chelpipe.ru/'
base_html = get_html(base_url)

# browser.implicitly_wait(0.5)
main_list = list()
a = '/production/truby-nerzhaveyushchie/'
page = 1
new_url = (base_url + a).replace('//', '/')
browser.get(new_url)
while True:
    try:
        table = browser.find_element_by_class_name('table__main')
        print(page)
    except:
        break

    table_html = table.get_attribute('outerHTML')
    table_soup = BeautifulSoup(table_html, 'lxml')
    div_row = table_soup.find_all('div', {'class': 'table-row'})
    for dr in div_row:
        page_product = dr.find('a').get('href')

        try:
          product = get_html(base_url + page_product[1:])
          product = BeautifulSoup(product, 'lxml')
        except common.exceptions.TimeoutException:
            print('TimeoutException')
            print((base_url + page_product).replace('//', '/'))
            product = get_html(base_url + page_product[1:])
            product = BeautifulSoup(product, 'lxml')
        try:
          product_page = product.find('div', {'class': 'product-card-main__stats'})
        except common.exceptions.NoSuchElementException:
          print('NoSuchElementException')
          print(browser.current_url)
          product_page = product.find('div', {'class': 'product-card-main__stats'})
        data = product_page.find_all('div', {'class': 'product-card-main__stats-item-value'})

        production_method = data[5].text
        if production_method == 'Горячедеформированные трубы':
          production_method = 'г/д'
        elif production_method == 'Холоднодеформированные трубы':
          production_method = 'х/д'
        elif production_method == 'Электросварные трубы':
          production_method = 'ЭСВ'
        else:
          production_method = None

        article = data[-1].text
        type_of_length = data[-2].text.strip()

        row = dr.find_all('div', {'class': 'table-row__item-value'})

        for_tube = dr.find('div', {'class': 'table-row__item-value-secondary'})
        type_tube = for_tube.find_all('div')[-1].text.strip()
        standard = dr.find('div', {'class': 'table-row__item-value-primary'}).text.split(',')
        steel = row[0].text
        diameter = float(row[1].text.split('/')[0].strip())
        wall = float(row[2].text)
        region = row[3].text
        a_r = row[4].find('span').text.split('/')

        try:
            availability = float(a_r[0].strip())
        except ValueError:
            availability = 0

        try:
            if len(a_r) > 1:
                reserve = float(a_r[-1].strip())
            else:
                reserve = 0

        except ValueError:
            raise ValueError()
        float_prices = []
        if 'от' in row[5].text:
          price_table =  product.find('div', {'class': "new-table__inner"})
          price_table = price_table.find_all('div', {'class': 'new-table__row'})[1]
          prices = price_table.find_all('div', {'class': 'new-table__cell--order-1'})
          for pr in prices:
            buff_price = pr.find('div', {'class': 'new-table__cell-value'}).text.replace(u'\xa0', u'').replace('руб', '').strip()
            if buff_price == '—':
              float_prices.append(0)
            else:
              float_prices.append(float(buff_price))
        else:
          try:
              price = float(row[5].text.replace(u'\xa0', u'').replace('i', '').replace('от', '').replace('руб.', '').strip())
          except ValueError:
              price = 0

        for s in standard:
            main_list.append({'type_tube': type_tube, 
                              'standard': s.replace('\n', '').strip(),
                              'steel': steel,
                              'diameter': diameter,
                              'wall': wall,
                              'region': region,
                              'availability': availability,
                              'reserve': reserve,
                              'price_min': float_prices[0] if float_prices else price,
                              'price_max': float_prices[-1] if float_prices else None,
                              'article': article,
                              'type_of_length': type_of_length,
                              'production_method': production_method})

    page += 1
    browser.get(new_url + f'?PAGEN_1={page}')

1
2
3
4
5
6


In [5]:
json_list = json.dumps(main_list,  ensure_ascii=False).replace('},', '},\n')
with open('truby-nerzhaveyushchie.json', "w",  encoding='utf-8') as file:
    file.write(json_list)
browser.quit()